The below codes perform dimensionality reduction for different number of hidden units of 2,5 ,10. In general the following observations were made:
1. The average recall and precision of autoencoder was greater than the corresponding PCA latent code based retrieval.
2. The average recall and the precision increased with the number of hidden units.



In [21]:
# Autoencoder with 2 hidden units
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
learning_rate = 0.01
num_steps = 30000
batch_size = 1

display_step = 1000
num_hidden_1 = 2 
num_input = 784 
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation 
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_1
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()
train_data, train_label = mnist.train.next_batch(20000)
int_enc =  [ np.where(r==1)[0][0] for r in train_label ]
new_train_data = []
new_train_label =[]
count = {}

#Reading only first 100 examples
for num in xrange(0,11):
    indices = [i for i, x in enumerate(int_enc) if x == num]
    indices = indices[0:100]
    for ind in indices:
        new_train_data.append(train_data[ind])
        new_train_label.append(int_enc[ind])

with tf.Session() as sess:
    sess.run(init)
    for iter in range(10):
        for i in range(len(new_train_data)):
            batch_x, label = [new_train_data[i]], [new_train_label[i]]
            _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
            # Display logs per step
            if i % display_step == 0 or i == 1:
                print('Step %i: Reconstruction Loss: %f' % (i, l))
    encoded_codes = []         
    for i in range(len(new_train_data)):
        enc = sess.run(encoder_op, feed_dict={X:[new_train_data[i]]})
#         print(enc)
        encoded_codes.append(enc[0])
    encoded_codes=np.array(encoded_codes)
    print(encoded_codes.shape)
    
X_transformed=encoded_codes
for test_query in range(X_transformed.shape[0]):
    correct = 0
    distances =[]
    for j in range(X_transformed.shape[0]):
        distances.append((new_train_label[j],np.linalg.norm(X_transformed[test_query] - X_transformed[j])))
    distances.sort(key=lambda x: x[1])
#     print(distances[1:10])
    labels = [i[0] for i in distances]
    labels = labels[1:50]
    correct+=labels.count(new_train_label[test_query])
    print("The recall for image {0} is {1}".format(test_query,correct/100))
    print("The precision for image {0} is {1}".format(test_query,correct/50))
    average_recall+=correct/100
    average_precision+=correct/50

print("Average Recall" , average_recall/1000)
print("Average Precision", average_precision/1000)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Step 0: Reconstruction Loss: 0.303290
Step 1: Reconstruction Loss: 0.297598
Step 0: Reconstruction Loss: 0.153111
Step 1: Reconstruction Loss: 0.079007
Step 0: Reconstruction Loss: 0.151816
Step 1: Reconstruction Loss: 0.078689
Step 0: Reconstruction Loss: 0.153725
Step 1: Reconstruction Loss: 0.079569
Step 0: Reconstruction Loss: 0.156164
Step 1: Reconstruction Loss: 0.078840
Step 0: Reconstruction Loss: 0.131588
Step 1: Reconstruction Loss: 0.079326
Step 0: Reconstruction Loss: 0.133940
Step 1: Reconstruction Loss: 0.074956
Step 0: Reconstruction Loss: 0.132917
Step 1: Reconstruction Loss: 0.074349
Step 0: Reconstruction Loss: 0.133977
Step 1: Reconstruction Loss: 0.074877
Step 0: Reconstruction Loss: 0.133883
Step 1: Reconstruction Loss: 0.074358
(1000, 2)
The recall for image 0 is 0.05
The preci

In [14]:
#PCA with 2 hidden units

import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.decomposition import RandomizedPCA
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from sklearn.metrics import label_ranking_average_precision_score
#use all digits
mnist = fetch_mldata("MNIST original")
X_train, y_train = new_train_data,new_train_label
pca = RandomizedPCA(n_components=2)
plt.prism()

X_transformed = pca.fit_transform(X_train)
distances = {}
print(X_transformed.shape)
average_precision = 0
average_recall = 0
for test_query in range(X_transformed.shape[0]):
    correct = 0
    distances =[]
    for j in range(X_transformed.shape[0]):
        distances.append((new_train_label[j],np.linalg.norm(X_transformed[test_query] - X_transformed[j])))
    distances.sort(key=lambda x: x[1])
#     print(distances[1:10])
    labels = [i[0] for i in distances]
    labels = labels[1:50]
    correct+=labels.count(new_train_label[test_query])
    print("The recall for image {0} is {1}".format(test_query,correct/100))
    print("The precision for image {0} is {1}".format(test_query,correct/50))
    average_recall+=correct/100
    average_precision+=correct/50

print("Average Recall" , average_recall/1000)
print("Average Precision", average_precision/1000)

    
    

/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:57: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


(1000, 2)
The recall for image 0 is 0.16
The precision for image 0 is 0.32
The recall for image 1 is 0.3
The precision for image 1 is 0.6
The recall for image 2 is 0.14
The precision for image 2 is 0.28
The recall for image 3 is 0.13
The precision for image 3 is 0.26
The recall for image 4 is 0.09
The precision for image 4 is 0.18
The recall for image 5 is 0.1
The precision for image 5 is 0.2
The recall for image 6 is 0.22
The precision for image 6 is 0.44
The recall for image 7 is 0.07
The precision for image 7 is 0.14
The recall for image 8 is 0.31
The precision for image 8 is 0.62
The recall for image 9 is 0.03
The precision for image 9 is 0.06
The recall for image 10 is 0.36
The precision for image 10 is 0.72
The recall for image 11 is 0.19
The precision for image 11 is 0.38
The recall for image 12 is 0.36
The precision for image 12 is 0.72
The recall for image 13 is 0.29
The precision for image 13 is 0.58
The recall for image 14 is 0.22
The precision for image 14 is 0.44
The recal

In [22]:
# Autoencoder with 5 hidden units
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
learning_rate = 0.01
num_steps = 30000
batch_size = 1

display_step = 1000
examples_to_show = 10
num_hidden_1 = 5
num_input = 784 
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_1
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()
train_data, train_label = mnist.train.next_batch(20000)
int_enc =  [ np.where(r==1)[0][0] for r in train_label ]
new_train_data = []
new_train_label =[]
count = {}
for num in xrange(0,11):
    indices = [i for i, x in enumerate(int_enc) if x == num]
    indices = indices[0:100]
    for ind in indices:
        new_train_data.append(train_data[ind])
        new_train_label.append(int_enc[ind])

with tf.Session() as sess:
    sess.run(init)
    for iter in range(10):
        for i in range(len(new_train_data)):
            batch_x, label = [new_train_data[i]], [new_train_label[i]]
            _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
            # Display logs per step
            if i % display_step == 0 or i == 1:
                print('Step %i: Reconstruction Loss: %f' % (i, l))
    encoded_codes = []         
    for i in range(len(new_train_data)):
        enc = sess.run(encoder_op, feed_dict={X:[new_train_data[i]]})
#         print(enc)
        encoded_codes.append(enc[0])
    encoded_codes=np.array(encoded_codes)
    print(encoded_codes.shape)
    
X_transformed=encoded_codes
for test_query in range(X_transformed.shape[0]):
    correct = 0
    distances =[]
    for j in range(X_transformed.shape[0]):
        distances.append((new_train_label[j],np.linalg.norm(X_transformed[test_query] - X_transformed[j])))
    distances.sort(key=lambda x: x[1])
#     print(distances[1:10])
    labels = [i[0] for i in distances]
    labels = labels[1:50]
    correct+=labels.count(new_train_label[test_query])
    print("The recall for image {0} is {1}".format(test_query,correct/100))
    print("The precision for image {0} is {1}".format(test_query,correct/50))
    average_recall+=correct/100
    average_precision+=correct/50

print("Average Recall" , average_recall/1000)
print("Average Precision", average_precision/1000)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Step 0: Reconstruction Loss: 0.319310
Step 1: Reconstruction Loss: 0.323335
Step 0: Reconstruction Loss: 0.062812
Step 1: Reconstruction Loss: 0.066176
Step 0: Reconstruction Loss: 0.062671
Step 1: Reconstruction Loss: 0.066717
Step 0: Reconstruction Loss: 0.061733
Step 1: Reconstruction Loss: 0.063164
Step 0: Reconstruction Loss: 0.059260
Step 1: Reconstruction Loss: 0.064369
Step 0: Reconstruction Loss: 0.058543
Step 1: Reconstruction Loss: 0.073294
Step 0: Reconstruction Loss: 0.057511
Step 1: Reconstruction Loss: 0.064765
Step 0: Reconstruction Loss: 0.059679
Step 1: Reconstruction Loss: 0.056918
Step 0: Reconstruction Loss: 0.050421
Step 1: Reconstruction Loss: 0.055397
Step 0: Reconstruction Loss: 0.055304
Step 1: Reconstruction Loss: 0.055752
(1000, 5)
The recall for image 0 is 0.05
The preci

In [23]:
#PCA with hidden units 5

import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.decomposition import RandomizedPCA
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from sklearn.metrics import label_ranking_average_precision_score
#use all digits
mnist = fetch_mldata("MNIST original")
X_train, y_train = new_train_data,new_train_label
pca = RandomizedPCA(n_components=5)
plt.prism()

X_transformed = pca.fit_transform(X_train)
distances = {}
print(X_transformed.shape)
average_precision = 0
average_recall = 0
for test_query in range(X_transformed.shape[0]):
    correct = 0
    distances =[]
    for j in range(X_transformed.shape[0]):
        distances.append((new_train_label[j],np.linalg.norm(X_transformed[test_query] - X_transformed[j])))
    distances.sort(key=lambda x: x[1])
#     print(distances[1:10])
    labels = [i[0] for i in distances]
    labels = labels[1:50]
    correct+=labels.count(new_train_label[test_query])
    print("The recall for image {0} is {1}".format(test_query,correct/100))
    print("The precision for image {0} is {1}".format(test_query,correct/50))
    average_recall+=correct/100
    average_precision+=correct/50

print("Average Recall" , average_recall/1000)
print("Average Precision", average_precision/1000)

    
    

/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:57: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


(1000, 5)
The recall for image 0 is 0.04
The precision for image 0 is 0.08
The recall for image 1 is 0.16
The precision for image 1 is 0.32
The recall for image 2 is 0.05
The precision for image 2 is 0.1
The recall for image 3 is 0.41
The precision for image 3 is 0.82
The recall for image 4 is 0.45
The precision for image 4 is 0.9
The recall for image 5 is 0.31
The precision for image 5 is 0.62
The recall for image 6 is 0.33
The precision for image 6 is 0.66
The recall for image 7 is 0.46
The precision for image 7 is 0.92
The recall for image 8 is 0.39
The precision for image 8 is 0.78
The recall for image 9 is 0.42
The precision for image 9 is 0.84
The recall for image 10 is 0.28
The precision for image 10 is 0.56
The recall for image 11 is 0.45
The precision for image 11 is 0.9
The recall for image 12 is 0.21
The precision for image 12 is 0.42
The recall for image 13 is 0.13
The precision for image 13 is 0.26
The recall for image 14 is 0.33
The precision for image 14 is 0.66
The reca

In [24]:
# Autoencoder with 10 hidden units
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
learning_rate = 0.01
num_steps = 30000
batch_size = 1

display_step = 1000
examples_to_show = 10
num_hidden_1 = 10
num_input = 784 
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_1
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()
train_data, train_label = mnist.train.next_batch(20000)
int_enc =  [ np.where(r==1)[0][0] for r in train_label ]
new_train_data = []
new_train_label =[]
count = {}
for num in xrange(0,11):
    indices = [i for i, x in enumerate(int_enc) if x == num]
    indices = indices[0:100]
    for ind in indices:
        new_train_data.append(train_data[ind])
        new_train_label.append(int_enc[ind])

with tf.Session() as sess:
    sess.run(init)
    for iter in range(10):
        for i in range(len(new_train_data)):
            batch_x, label = [new_train_data[i]], [new_train_label[i]]
            _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
            # Display logs per step
            if i % display_step == 0 or i == 1:
                print('Step %i: Reconstruction Loss: %f' % (i, l))
    encoded_codes = []         
    for i in range(len(new_train_data)):
        enc = sess.run(encoder_op, feed_dict={X:[new_train_data[i]]})
#         print(enc)
        encoded_codes.append(enc[0])
    encoded_codes=np.array(encoded_codes)
    print(encoded_codes.shape)
    
X_transformed=encoded_codes
for test_query in range(X_transformed.shape[0]):
    correct = 0
    distances =[]
    for j in range(X_transformed.shape[0]):
        distances.append((new_train_label[j],np.linalg.norm(X_transformed[test_query] - X_transformed[j])))
    distances.sort(key=lambda x: x[1])
#     print(distances[1:10])
    labels = [i[0] for i in distances]
    labels = labels[1:50]
    correct+=labels.count(new_train_label[test_query])
    print("The recall for image {0} is {1}".format(test_query,correct/100))
    print("The precision for image {0} is {1}".format(test_query,correct/50))
    average_recall+=correct/100
    average_precision+=correct/50

print("Average Recall" , average_recall/1000)
print("Average Precision", average_precision/1000)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Step 0: Reconstruction Loss: 0.302404
Step 1: Reconstruction Loss: 0.311896
Step 0: Reconstruction Loss: 0.122658
Step 1: Reconstruction Loss: 0.056992
Step 0: Reconstruction Loss: 0.104793
Step 1: Reconstruction Loss: 0.051022
Step 0: Reconstruction Loss: 0.091895
Step 1: Reconstruction Loss: 0.045729
Step 0: Reconstruction Loss: 0.085067
Step 1: Reconstruction Loss: 0.048158
Step 0: Reconstruction Loss: 0.081242
Step 1: Reconstruction Loss: 0.045540
Step 0: Reconstruction Loss: 0.077721
Step 1: Reconstruction Loss: 0.045635
Step 0: Reconstruction Loss: 0.085078
Step 1: Reconstruction Loss: 0.047335
Step 0: Reconstruction Loss: 0.073624
Step 1: Reconstruction Loss: 0.048584
Step 0: Reconstruction Loss: 0.076444
Step 1: Reconstruction Loss: 0.046247
(1000, 10)
The recall for image 0 is 0.17
The prec

In [25]:
#PCA with hidden units 10

import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.decomposition import RandomizedPCA
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from sklearn.metrics import label_ranking_average_precision_score
#use all digits
mnist = fetch_mldata("MNIST original")
X_train, y_train = new_train_data,new_train_label
pca = RandomizedPCA(n_components=10)
plt.prism()

X_transformed = pca.fit_transform(X_train)
distances = {}
print(X_transformed.shape)
average_precision = 0
average_recall = 0
for test_query in range(X_transformed.shape[0]):
    correct = 0
    distances =[]
    for j in range(X_transformed.shape[0]):
        distances.append((new_train_label[j],np.linalg.norm(X_transformed[test_query] - X_transformed[j])))
    distances.sort(key=lambda x: x[1])
#     print(distances[1:10])
    labels = [i[0] for i in distances]
    labels = labels[1:50]
    correct+=labels.count(new_train_label[test_query])
    print("The recall for image {0} is {1}".format(test_query,correct/100))
    print("The precision for image {0} is {1}".format(test_query,correct/50))
    average_recall+=correct/100
    average_precision+=correct/50

print("Average Recall" , average_recall/1000)
print("Average Precision", average_precision/1000)

    
    

/home/subha/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:57: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


(1000, 10)
The recall for image 0 is 0.46
The precision for image 0 is 0.92
The recall for image 1 is 0.07
The precision for image 1 is 0.14
The recall for image 2 is 0.39
The precision for image 2 is 0.78
The recall for image 3 is 0.43
The precision for image 3 is 0.86
The recall for image 4 is 0.32
The precision for image 4 is 0.64
The recall for image 5 is 0.27
The precision for image 5 is 0.54
The recall for image 6 is 0.41
The precision for image 6 is 0.82
The recall for image 7 is 0.26
The precision for image 7 is 0.52
The recall for image 8 is 0.26
The precision for image 8 is 0.52
The recall for image 9 is 0.37
The precision for image 9 is 0.74
The recall for image 10 is 0.35
The precision for image 10 is 0.7
The recall for image 11 is 0.03
The precision for image 11 is 0.06
The recall for image 12 is 0.32
The precision for image 12 is 0.64
The recall for image 13 is 0.21
The precision for image 13 is 0.42
The recall for image 14 is 0.26
The precision for image 14 is 0.52
The r